# change to load data from load_data.py


data source: ./simplified_data/simplified_data2.csv

data source: ./simplified_data/simplified_data2.csv

data 處理:

data v2 刪掉所有文字input

deleted 回診,西藥

## train 方法:

.....



In [1]:

from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from keras.callbacks import EarlyStopping, LambdaCallback
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from sklearn.utils.class_weight import compute_sample_weight




In [2]:
import numpy as np
import pandas as pd
import random
import json
import statistics
from tabulate import tabulate

# custom import 
import my_utilities as myutil
import load_data

## Read Data

1. 決定是否要delete 少於某threshold的藥
2. 用load_data裡的`load_data_for_n_med` load data
3. 如只train 頭n 個藥-> set `only_train_1st_n`

In [3]:
del_med_under_thres = 50     # 於所有medical cases中出現次數少於此數的藥->整col 刪除
                            # if set to 250 -> will leave 10 medicine

total_med = 102
only_train_1st_n = total_med

(X_np, X_val_np, train_y, val_y, 
  num_col_x, num_1_valy, num_0_valy) = load_data.load_data_for_1_med_with_debug(del_med_thres=del_med_under_thres, n=only_train_1st_n)

assert(isinstance(X_np, np.ndarray))
assert(isinstance(X_val_np, np.ndarray))
assert(isinstance(train_y, pd.DataFrame))
assert(isinstance(val_y, pd.DataFrame))
assert(isinstance(num_col_x, int))


in ReadData
type of data: <class 'pandas.core.frame.DataFrame'>
Shape of data = (797 rows, 215 cols).
SplitXY:
Shape of X = (796 rows, 111 cols). First 10 data of X:
    肺癌  胰臟癌  肝癌  腺癌  攝護腺癌  骨癌  淋巴癌  胃癌  腦瘤  肝炎
1    0    0   0   0     0   0    0   0   0   0
2    0    0   0   0     0   0    0   0   0   0
3    0    0   0   0     0   0    0   0   0   0
4    0    0   0   0     0   0    0   0   0   0
5    0    0   0   0     0   0    0   0   0   0
6    0    0   0   0     0   0    0   0   0   0
7    0    0   0   0     0   0    0   0   0   0
8    0    0   0   0     0   0    0   0   0   0
9    0    0   0   0     0   0    0   0   0   0
10   0    0   0   0     0   0    0   0   0   0
Shape of y = (796 rows, 102 cols). First 10 data of y:
    麻黃  桂枝  荊芥  防風  細辛  白芷  生薑  辛夷  葛根  升麻
1    0   1   0   0   1   1   0   0   0   0
2    0   1   0   0   0   0   1   0   0   0
3    0   0   0   0   0   0   0   0   0   0
4    0   1   0   0   0   0   1   1   0   0
5    0   1   0   0   0   0   0   0   0   0
6   

In [7]:
# data type checking  you can run this if you suspect data type 
# else can skip this cell


# myutil.print_df(val_y, "---- y ----")
# print(val_y)
# checking
# 1. Count occurrence of na, int, float, str in y
na_count = val_y.isna().sum().sum()
str_count = val_y[val_y.map(type) == str].count().sum()
int_count = val_y[val_y.map(type) == int].count().sum()
float_count = val_y[val_y.map(type) == float].count().sum()

print(f"Number of NA values in y: {na_count}")
print(f"Number of str values in y: {str_count}")
print(f"Number of int values in y: {int_count}")
print(f"Number of float values in y: {float_count}")

Number of NA values in y: 0
Number of str values in y: 0
Number of int values in y: 7155
Number of float values in y: 0


## compute class weight

In [5]:

# # class_weights = compute_sample_weight(class_weight='balanced', y=train_y)
# # class_weight_dict = dict(enumerate(class_weights))

# # print(class_weight_dict)

# train_y_np = np.array(train_y)

# num_labels = train_y_np.shape[1]
# class_weight_dic = {}

# for i in range(num_labels):
#     unique_values, counts = np.unique(train_y_np[:, i], return_counts=True)
#     value_frequency_dict = dict(zip(unique_values, counts))
#     total = value_frequency_dict[0] + value_frequency_dict[1]
#     class_weight_dic[i] = {0: (value_frequency_dict[1] / total), 1: 5 * (value_frequency_dict[0] / total)} 
#     # f1: 0.35
#     # class_weight_dic[i] = {0: (total / value_frequency_dict[0]), 1: 5 * (total / value_frequency_dict[1])}

# print(class_weight_dic)
# class_weight_list = np.array(class_weight_dic)

## Build Model

In [8]:
model = Sequential([
    Dense (units=64, input_shape=(num_col_x,), activation='sigmoid'),
    Dropout(0.1),
    Dense (units=128, activation='softmax'), 
    Dropout(0.1),
    Dense (units=64, activation='softmax'), 
    Dropout(0.1),
    Dense (units=32, activation='softmax'), 
    Dense (units=2, activation='softmax')
])
model.summary()

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 64)                7168      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_6 (Dense)             (None, 128)               8320      
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_7 (Dense)             (None, 64)                8256      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_8 (Dense)             (None, 32)               

## Train Model

train medicine-by-medicine

In [9]:
all_res=[]
result_df_dict={}   # dict of df
#loss_acc_dict={}    # dict of loss and accuracy of each medicine
accuracy_dict={}    # dict of accuracy of each medicine
raw_prediction_train_dict={}   # dict of raw prediction of train set
raw_prediction_val_dict={}   # dict of raw prediction of val set

for i in range(train_y.shape[1]):
#for i in range(2):
    res = {}
    res_list =[]
    chosen_col = train_y.iloc[:,i].copy()
    assert(isinstance(chosen_col, pd.Series))
    assert(len(chosen_col) == len(train_y))
    print(f"processing the {i} of {train_y.shape[1]} medicine: { chosen_col.name}")

    chosen_y_np = chosen_col.values.astype('float64')

    y_val_chosen_col = val_y.iloc[:,i].copy()
    assert(isinstance(y_val_chosen_col, pd.Series))
    assert(len(y_val_chosen_col) == len(val_y))
    #y_val_chosen_col_np = y_val_chosen_col.values.astype('float64')

    
    # Early stop
    early_stopping = EarlyStopping(monitor='loss', patience=100, restore_best_weights=True)

    
    # fit model for this medicine
    model.fit(
        x=X_np,
        y=chosen_y_np,
        # class_weight=class_weight_dic[i],
        epochs=2000,
        shuffle=True,
        verbose=0,
        callbacks=[early_stopping]
    )   # batch_size=32 if not specified
    
    # make prediction for this medicine
    predictions_val_set = model.predict(X_val_np)
    if(i==1):
        print("type of predictions val: " ,type(predictions_val_set))
        print("predictions.shape:", predictions_val_set.shape)
        
    raw_prediction_val_dict[chosen_col.name] = predictions_val_set  # save raw result np of val set to dict
    

    predictions_train_set = model.predict(X_np)   # predict against training set for diagonse overfit or underfit
    raw_prediction_train_dict[chosen_col.name] = predictions_train_set  # save raw result np of train set to dict

print("training done.")

# to do : calu TP, FP, TN, FN of train set like val set


processing medicine: 麻黃
5/5 [==============================] - 0s 3ms/step
<class 'numpy.ndarray'>
predictions.shape: (159, 2)
processing medicine: 桂枝
5/5 [==============================] - 0s 4ms/step
<class 'numpy.ndarray'>
predictions.shape: (159, 2)
processing medicine: 細辛
5/5 [==============================] - 0s 4ms/step
<class 'numpy.ndarray'>
predictions.shape: (159, 2)
processing medicine: 生薑
5/5 [==============================] - 0s 4ms/step
<class 'numpy.ndarray'>
predictions.shape: (159, 2)
processing medicine: 柴胡
5/5 [==============================] - 0s 4ms/step
<class 'numpy.ndarray'>
predictions.shape: (159, 2)
processing medicine: 石膏
5/5 [==============================] - 0s 4ms/step
<class 'numpy.ndarray'>
predictions.shape: (159, 2)
processing medicine: 知母
5/5 [==============================] - 0s 3ms/step
<class 'numpy.ndarray'>
predictions.shape: (159, 2)
processing medicine: 梔子
5/5 [==============================] - 0s 4ms/step
<class 'numpy.ndarray'>
predictions.

## Handle result

In [ ]:
for key, arr in raw_prediction_val_dict.items():
    df_predictions = pd.DataFrame(arr, columns=["predicted as 0", "predicted as 1"])
    df_predictions["predicted value"] = np.where(df_predictions["predicted as 0"] > df_predictions["predicted as 1"], 0, 1)
    col_num = val_y.columns.get_loc(key)
    df_predictions["ground truth"] = val_y.iloc[:,col_num].copy().values
    df_predictions["is_correct"] = df_predictions["predicted value"] == df_predictions["ground truth"]
    accuracy = df_predictions["is_correct"].mean()
    accuracy_dict[key] = accuracy  
    result_df_dict[key] = df_predictions
    result_df_dict[key] = df_predictions

In [10]:

predicted_value_index = None
ground_truth_index = None

acc_each_med={}
total_tp=0
total_fp=0
total_fn=0
total_tn=0

# df.columns.tolist()= pick list of column names

for i in train_y.columns.tolist(): # build empty dict for holding TP, FP, FN, TN, accuracy, f1_score of each medicine
    acc_each_med[i]={"TP": None, "FP": None, "FN": None, "TN": None, "accuracy": None, "f1_score": None}

# calculate f1 score of for each medicine

for key, df in result_df_dict.items():
    # create 'TP/FP/TN/TN/FN' column

    if predicted_value_index is None:
        predicted_value_index = df.columns.get_loc('predicted value')
    if ground_truth_index is None:
        ground_truth_index = df.columns.get_loc('ground truth')


    df['TP/FP/TN/TN/FN'] = df.apply(lambda row: 'TP' if ( row.iloc[predicted_value_index] and row.iloc[ground_truth_index] ) else 
                                 ('FP' if (row.iloc[predicted_value_index]  and (not row.iloc[ground_truth_index] )) else 
                                  ('FN' if (( not row.iloc[predicted_value_index] ) and row.iloc[ground_truth_index] ) else 
                                   ( 'TN' if ((not row.iloc[predicted_value_index]) and (not row.iloc[ground_truth_index]) ) else 'UN'))), axis=1)

    # count occurrences of 'TP', 'FP', 'FN', and 'TN' of *this medicine*
    counts = df['TP/FP/TN/TN/FN'].value_counts()
    # get number of 'TP', 'FP', 'FN', and 'TN'
    num_tp = counts.get('TP', 0)
    num_fp = counts.get('FP', 0)
    num_fn = counts.get('FN', 0)
    num_tn = counts.get('TN', 0)

    f1_score = 2 * num_tp / (2 * num_tp + num_fp + num_fn) if (2 * num_tp + num_fp + num_fn) > 0 else 0
    acc_each_med[key]["TP"] = num_tp
    acc_each_med[key]["FP"] = num_fp
    acc_each_med[key]["FN"] = num_fn
    acc_each_med[key]["TN"] = num_tn
    acc_each_med[key]["accuracy"] = accuracy_dict[key].item()
    acc_each_med[key]["f1_score"] = f1_score

    total_tp += num_tp
    total_fp += num_fp
    total_fn += num_fn
    total_tn += num_tn

    ### debug messages
    print("in for loop of result_df_dict")
    print(f"processing : {key} ....")
    print(f"TP of {key}: {num_tp}", sep="\t")
    print(f"FP of {key}: {num_fp}", sep="\t")
    print(f"FN of {key}: {num_fn}", sep="\t")
    print(f"TN of {key}: {num_tn}", sep="\t")
    print(f"f1_score of {key}: {f1_score}")
    print(f"total tp now is: {total_tp}"  , sep="\t")
    print(f"total fp now is: {total_fp}"  , sep="\t")
    print(f"total fn now is: {total_fn}"  , sep="\t")
    print(f"total tn now is: {total_tn}"  , sep="\t")

overall_f1 = 2 * total_tp / (2 * total_tp + total_fp + total_fn) if (2 * total_tp + total_fp + total_fn) > 0 else 0
total_med_case =  len(val_y)* val_y.shape[1]
print(" \n*****   end of for loop   *****\n")
print("now doing calculation checking...")
mean_accuracy = statistics.mean(accuracy_dict.values())
acc_each_med["overall"]={}
acc_each_med["overall"]["TP"] = total_tp
acc_each_med["overall"]["FP"] = total_fp
acc_each_med["overall"]["FN"] = total_fn
acc_each_med["overall"]["TN"] = total_tn
acc_each_med["overall"]["TP_percentage"] ={total_tp/total_med_case}
acc_each_med["overall"]["FP_percentage"] ={total_fp/total_med_case}
acc_each_med["overall"]["FN_percentage"] ={total_fn/total_med_case}
acc_each_med["overall"]["TN_percentage"] ={total_tn/total_med_case}
acc_each_med["overall"]["f1_score"]=overall_f1
acc_each_med["overall"]["mean_accuracy"]=mean_accuracy


try:
    assert((total_tp + total_fp + total_fn + total_tn) == total_med_case)
    assert((total_tp + total_fn)== num_1_valy)  # TP+FN=all 1 in val_y
    assert((total_fp + total_tn)== num_0_valy)  # TN+FP=all 0 in val_y
    
except:
    print("wrong calculation!")
    print(f"otal_tp + total_fp + total_fn + total_tn= {total_tp + total_fp + total_fn + total_tn}", sep="\t")
    print(f"total_med_case= {total_med_case}")
    print(f"total_tp + total_fn= {total_tp + total_fn}", sep="\t")
    print(f"num_1_valy= {num_1_valy}")
    print(f"total_fp + total_tn= {total_fp + total_tn}", sep="\t")
    print(f"num_0_valy= {num_0_valy}")
else:
    print("checking passed")


in for loop of result_df_dict
processing : 麻黃 ....
TP of 麻黃: 3
FP of 麻黃: 20
FN of 麻黃: 23
TN of 麻黃: 113
f1_score of 麻黃: 0.12244897959183673
total tp now is: 3
total fp now is: 20
total fn now is: 23
total tn now is: 113
in for loop of result_df_dict
processing : 桂枝 ....
TP of 桂枝: 33
FP of 桂枝: 31
FN of 桂枝: 37
TN of 桂枝: 58
f1_score of 桂枝: 0.4925373134328358
total tp now is: 36
total fp now is: 51
total fn now is: 60
total tn now is: 171
in for loop of result_df_dict
processing : 細辛 ....
TP of 細辛: 18
FP of 細辛: 20
FN of 細辛: 39
TN of 細辛: 82
f1_score of 細辛: 0.37894736842105264
total tp now is: 54
total fp now is: 71
total fn now is: 99
total tn now is: 253
in for loop of result_df_dict
processing : 生薑 ....
TP of 生薑: 10
FP of 生薑: 20
FN of 生薑: 35
TN of 生薑: 94
f1_score of 生薑: 0.26666666666666666
total tp now is: 64
total fp now is: 91
total fn now is: 134
total tn now is: 347
in for loop of result_df_dict
processing : 柴胡 ....
TP of 柴胡: 20
FP of 柴胡: 19
FN of 柴胡: 45
TN of 柴胡: 75
f1_score of 柴胡: 0.

In [11]:


print(f"total tp: {total_tp}, \t percentage = {total_tp/total_med_case}" )
print(f"total fp: {total_fp}, \t percentage = {total_fp/total_med_case}" )
print(f"total fn: {total_fn}, \t percentage = {total_fn/total_med_case}" )
print(f"total tn: {total_tn}, \t percentage = {total_tn/total_med_case}" )
print(f"overall f1 score: {overall_f1}")
print("mean accuracy of all medicine: ", mean_accuracy)
print(acc_each_med)         # can't use json.dumps as there are np.int64
# precision = 判斷為true之中有多少是對的 = TP / (TP + FP) = 0.4
# recall  =  實際為true之中有多少被找到  = TP / (TP + FN) = 0.28


total tp: 464, 	 percentage = 0.06484975541579314
total fp: 616, 	 percentage = 0.08609364081062194
total fn: 1179, 	 percentage = 0.16477987421383647
total tn: 4896, 	 percentage = 0.6842767295597484
overall f1 score: 0.34080058758722
mean accuracy of all medicine:  0.7491264849755416
{'麻黃': {'TP': 3, 'FP': 20, 'FN': 23, 'TN': 113, 'accuracy': 0.7295597484276729, 'f1_score': 0.12244897959183673}, '桂枝': {'TP': 33, 'FP': 31, 'FN': 37, 'TN': 58, 'accuracy': 0.5723270440251572, 'f1_score': 0.4925373134328358}, '細辛': {'TP': 18, 'FP': 20, 'FN': 39, 'TN': 82, 'accuracy': 0.6289308176100629, 'f1_score': 0.37894736842105264}, '生薑': {'TP': 10, 'FP': 20, 'FN': 35, 'TN': 94, 'accuracy': 0.6540880503144654, 'f1_score': 0.26666666666666666}, '柴胡': {'TP': 20, 'FP': 19, 'FN': 45, 'TN': 75, 'accuracy': 0.5974842767295597, 'f1_score': 0.38461538461538464}, '石膏': {'TP': 3, 'FP': 15, 'FN': 12, 'TN': 129, 'accuracy': 0.8301886792452831, 'f1_score': 0.18181818181818182}, '知母': {'TP': 2, 'FP': 19, 'FN': 18,

In [12]:

file_path_suffix = "each_med_csv"   # type the dir for you to remember where u save the result
for key, df in result_df_dict.items():
    print(f"DataFrame for {key}:")
    myutil.print_df(df)
    myutil.df_to_csv(df, save_path=("./result/"+file_path_suffix), file_prefix=key)
   



DataFrame for 麻黃:

+-----+------------------+------------------+-------------------+----------------+--------------+------------------+
|     |   predicted as 0 |   predicted as 1 |   predicted value |   ground truth | is_correct   | TP/FP/TN/TN/FN   |
|-----+------------------+------------------+-------------------+----------------+--------------+------------------|
|   0 |           0.9642 |           0.0358 |                 0 |              0 | True         | TN               |
|   1 |           0.9641 |           0.0359 |                 0 |              0 | True         | TN               |
|   2 |           0.9641 |           0.0359 |                 0 |              0 | True         | TN               |
|   3 |           0.8983 |           0.1017 |                 0 |              0 | True         | TN               |
|   4 |           0.9642 |           0.0358 |                 0 |              0 | True         | TN               |
|   5 |           0.9642 |           0.0358 |

In [13]:
# save f1_score and TP/FP/TN/TN/FN

spec_str = "del_med_under_thres = " + str(del_med_under_thres) +"epochs=2000, shuffle=True, verbose=0, callbacks=[early_stopping], learning_rate=0.001, layer=[64, 128, 64, 32, 2], activation='sigmoid', 'softmax', 'softmax', 'softmax', 'softmax'"
# need to type this spec str each time to record the result

file_path="./result/1_med_accuracy"
myutil.dict_to_txt(acc_each_med, save_path=file_path, 
                   file_prefix="accuracy_each_med",
                   textbox=spec_str)

accuracy_each_med saved to ./result/1_med_accuracy/accuracy_each_med_3.txt
